In [17]:
from supriya import *
server = Server()

In [18]:
server.boot()

<Server: udp://127.0.0.1:57751, 8i8o>

In [19]:
server.is_running

True

## We want to build a synthdef that will output to whatever a buffer of a fixed length, then we want to write to that buffer, and then play back that buffer

In [20]:
from supriya.ugens import *
sr = 44.1 * 1000
seconds = 100
nframes = sr * seconds
b = Buffer()
b = b.allocate(frame_count=nframes,
               channel_count=2)

In [21]:
def newSynthDefBufWr(buffer):
    with SynthDefBuilder() as builder:
        phase = Phasor.ar(rate = BufRateScale.kr(buffer_id=buffer.buffer_id),
                          start = 0,
                          stop = BufFrames.kr(buffer_id=buffer.buffer_id))
        source = SoundIn.ar(bus=(0,1))
#         source = (SinOsc.ar(220), SinOsc.ar(220*(3/4)))
        source = BufWr.ar(
            buffer_id=b.buffer_id,
            loop=0,
            phase=phase,
            source=source
        )
        out = Out.ar(bus=(4,5), source=source)
    synthdef = builder.build().allocate()
    server.sync()
    return synthdef

In [22]:
bufwr = newSynthDefBufWr(b)

In [83]:
# def newSynthDefBufRd(buffer, end):
with SynthDefBuilder(buffer = 0,
                     rate = 1, 
                     trig = 0,
                     loop=0,
                     start=0,
                     end=1) as builder:
    buffer = builder['buffer']
#     phase = Line.ar(start=0, stop=builder['end'], duration=builder['rate'])
    phase = Phasor.ar(rate = builder['rate']*BufRateScale.kr(buffer_id=buffer),
                     start = builder['start'],
                     stop = builder['end'], trigger = [SinOsc.ar(frequency=2)]
                     )
#                           reset_pos = 0,
#                           trigger = builder['trig'])
    source = BufRd.ar(
        buffer_id=buffer,
        channel_count=2,
        interpolation=2,
        loop=0,#builder['loop'],
        phase=[phase]*2
    )
    out = Out.ar(bus=(0,1), source=source)
    bufrd = builder.build().allocate()
    server.sync()
#     return synthdef

In [24]:
group = Group()
group = group.allocate()

In [25]:
import time
wrsynth = Synth(bufwr)

In [26]:
# start recording
start_time = time.time()
group.append(wrsynth)

In [27]:
# Stop recording
end_time = time.time()
wrsynth.free()

In [48]:
nseconds = end_time - start_time
nframes = round(b.sample_rate * nseconds)

**TODO**: here we probably want to copy original buffer to new buffer of exact length

In [85]:
# Play back recording
# bufrd = newSynthDefBufRd(b, round(b.sample_rate * (end_time - start_time)))
rdsynth = Synth(bufrd, rate=1, loop=0,buffer=b.buffer_id, start = 0, end=nframes)
group.append(rdsynth)

In [86]:
rdsynth.free()

TIMED OUT: QuitRequest()


In [61]:
rdsynth['start'] = round(nframes / 2.)

In [165]:
# Stop playing recording
rdsynth.free()

In [92]:
rdsynth['trig'] = 0
rdsynth['trig'] = 1

In [25]:
group.free()

<Group: ???>

In [33]:
rdsynth.free()